In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Bidirectional, Input, Embedding, Activation, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K
import pickle
import numpy as np
from model import dp_syllabler

In [9]:
#ortho max encoder len = 45
#ipa max encoder len = 74

with open('data/e2i_ortho.pkl', 'rb') as file:
    e2i_ortho = pickle.load(file)
    
with open('data/e2i_ipa.pkl', 'rb') as file:
    e2i_ipa = pickle.load(file)

shitter = dp_syllabler(e2i_ortho= e2i_ortho, e2i_ipa = e2i_ipa, ortho_input_size=45,ipa_input_size=74,latent_dim=32,embed_dim=32 ,max_feat=148)

In [10]:
def to_categorical(sequences):
        cat_sequences = []
        for s in sequences:
            cats = []
            for item in s:
                cats.append(np.zeros(3))
                cats[-1][item] = 1.0
            cat_sequences.append(cats)
        return np.array(cat_sequences)

In [11]:
with open('data/x_tr_ortho.pkl', 'rb') as file:
    x_tr_ortho = pickle.load(file)
    # x_tr_ortho = x_tr_ortho.reshape(len(x_tr_ortho), 45, 1)
    
# with open('data/x_tr_ipa.pkl', 'rb') as file:
#     x_tr_ipa = pickle.load(file)
#     # x_tr_ipa = x_tr_ipa.reshape(len(x_tr_ipa), 74, 1)

with open('data/false_ipa.pkl', 'rb') as file:
    x_tr_ipa = pickle.load(file)
    # x_tr_ipa = x_tr_ipa.reshape(len(x_tr_ipa), 74, 1)
    
with open('data/y_tr.pkl', 'rb') as file:
    y_tr = pickle.load(file)
    y_tr = y_tr.reshape(len(y_tr), 74, 1)
    
y_tr = to_categorical(y_tr)
split_index = int(.8 * len(x_tr_ortho))

x_test_ortho = x_tr_ortho[split_index:]
x_test_ipa = x_tr_ipa[split_index:]
x_test_ipa = x_test_ipa.reshape(len(x_test_ipa), 74, 1)
y_test = y_tr[split_index:]

x_tr_ortho = x_tr_ortho[:split_index]
x_tr_ipa = x_tr_ipa[:split_index]
y_tr = y_tr[:split_index]

In [ ]:
print(len(x_tr_ipa))
print(len(x_tr_ortho))
print(len(y_tr))

In [15]:
shitter.fit(x_tr_ortho, x_tr_ipa, y_tr, x_test_ortho, x_test_ipa, y_test, ep=50, batch_size=128, save_filename='data/32_false_double_pen_best_weights.h5')

Epoch 1/50
63/63 [==============================] - 95s 1s/step - loss: 0.0223 - accuracy: 0.9917 - ignore_accuracy: 0.9177 - val_loss: 0.0229 - val_accuracy: 0.9915 - val_ignore_accuracy: 0.9175

Epoch 00001: val_ignore_accuracy improved from -inf to 0.91753, saving model to data\32_false_double_pen_best_weights.h5
Epoch 2/50
63/63 [==============================] - 63s 997ms/step - loss: 0.0215 - accuracy: 0.9918 - ignore_accuracy: 0.9188 - val_loss: 0.0224 - val_accuracy: 0.9916 - val_ignore_accuracy: 0.9180

Epoch 00002: val_ignore_accuracy improved from 0.91753 to 0.91797, saving model to data\32_false_double_pen_best_weights.h5
Epoch 3/50
63/63 [==============================] - 63s 998ms/step - loss: 0.0209 - accuracy: 0.9921 - ignore_accuracy: 0.9217 - val_loss: 0.0220 - val_accuracy: 0.9919 - val_ignore_accuracy: 0.9206

Epoch 00003: val_ignore_accuracy improved from 0.91797 to 0.92060, saving model to data\32_false_double_pen_best_weights.h5
Epoch 4/50
63/63 [================

In [ ]:
shitter.model.load_weights('data/false_double_pen_best_weights.h5')

In [ ]:
test_ortho = ''.join(back_to_eng(x_test_ortho[7]))
test_ipa = ''.join(back_to_ipa(x_test_ipa[7]))

In [26]:
print(shitter.syllabify('tinkle', ippa))

tink-le


In [25]:
ippa = get_naive('tinkle')

In [16]:
l2i = {'a':'æ', 'b':'b', 'c':'k', 'd':'d', 'e':'e', 'f':'f' , 'g':'g', 'h':'h', 'i':'ɪ', 'j':'ʒ', 'k':'k', 'l':'l', 'm':'m', 'n':'n',
      'o':'ɑ', 'p':'p', 'q':'k', 'r':'ɹ', 's':'s', 't':'t', 'u':'ə', 'v':'v', 'w':'w', 'x':['k','s'], 'y':'j', 'z':'z'}

def get_naive(word):
    naive = []
    for c in word:
        naive += l2i[c]
    naive = ''.join(naive)
    return naive